# Testing Notebook

This notebook is used for running development version of the Vortector

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%reload_ext autoreload
%autoreload 2

from src.vortector.vortector import Vortector
from src.vortector.visualize import show_fit_overview_2D, show_fit_overview_1D
from src.vortector.contour import detect_elliptic_contours

%matplotlib widget

In [2]:
from opensimplex import OpenSimplex
def simplex_noise(Nx, Ny, seed=0, feature_size=None):
    if feature_size is None:
        feature_size = max(Nx, Ny)/13
    ng = OpenSimplex(seed)
    noise = np.zeros((Nx, Ny))
    for nx in range(Nx):
        for ny in range(Ny):
            noise[nx, ny] = ng.noise3d(nx/feature_size, ny/feature_size, 0)
    return noise

In [3]:
from numpy.random import default_rng
rng = default_rng(None)
rng.uniform(low=1, high=20)

19.79480138614315

### Creating a grid

In [4]:
# grid coordinates, use squared cells
ris = np.geomspace(1, 20, 122) # cell interfaces
phiis = np.linspace(-np.pi, np.pi, 252) # cell interfaces

# calculate cell centers, sizes
drs = ris[1:] - ris[:-1]
rs = 0.5*(ris[1:] + ris[:-1])
dphis = phiis[1:] - phiis[:-1]
phis = 0.5*(phiis[1:] + phiis[:-1])

# calculate cell area along the radial direction
area = dphis[0] * 0.5* (ris[1:]**2 - ris[:-1]**2)

# create meshgrids
Phis, Rs = np.meshgrid(phis, rs)
print("Rs.shape", Rs.shape)
print("Phis.shape", Phis.shape)

# calculate the area of the cells
Area = np.tile(area, (Rs.shape[1],1)).T
print("Area.shape", Area.shape)

Rs.shape (121, 251)
Phis.shape (121, 251)
Area.shape (121, 251)


In [5]:
r_vort = 10
r_vort_sigma = 2
phi_vort = np.pi/4
phi_vort_sigma = np.pi/8

vorticity = 1 - 0.8*np.exp( - 0.5*(Rs-r_vort)**2/r_vort_sigma**2)*np.exp( - 0.5*(Phis-phi_vort)**2/phi_vort_sigma**2)
density = 2 + 3*np.exp( - 0.5*(Rs-r_vort)**2/r_vort_sigma**2)*np.exp( - 0.5*(Phis-phi_vort)**2/phi_vort_sigma**2)

r_vort = 14
r_vort_sigma = 1
phi_vort = -np.pi/3
phi_vort_sigma = np.pi/8

vorticity -= 0.8*np.exp( - 0.5*(Rs-r_vort)**2/r_vort_sigma**2)*np.exp( - 0.5*(Phis-phi_vort)**2/phi_vort_sigma**2)
density += 2*np.exp( - 0.5*(Rs-r_vort)**2/r_vort_sigma**2)*np.exp( - 0.5*(Phis-phi_vort)**2/phi_vort_sigma**2)

vorticity += 0.1*simplex_noise(Rs.shape[0], Rs.shape[1], seed=3546384)
density += 0.5*simplex_noise(Rs.shape[0], Rs.shape[1], seed=6468946)

vortensity = vorticity/density
vortensity = vortensity/np.max(vortensity)

vd = Vortector(Rs, Phis, Area, vortensity, density, verbose=True)

In [51]:
from synthvortex import VortexGenerator
vgen = VortexGenerator(2, periodic_y=True)

# fig, axes = plt.subplots(1,2,dpi=150)

# im = axes[0].pcolormesh(vgen.Rs, vgen.Phis, vgen.vortensity, vmin=0, vmax=1, shading="auto")
# axes[0].contour(vgen.Rs, vgen.Phis, vgen.vortensity, cmap="Greys", linewidths=1)
# axes[0].set_xlabel("r")
# axes[0].set_ylabel("phi")
# cbar = fig.colorbar(im, ax=axes[0], orientation="horizontal")
# cbar.set_label("vortensity")

# im = axes[1].pcolormesh(vgen.Rs, vgen.Phis, vgen.density, shading="auto")
# axes[1].contour(vgen.Rs, vgen.Phis, vgen.density, cmap="Greys", linewidths=1)
# axes[1].set_xlabel("r")
# cbar = fig.colorbar(im, ax=axes[1], orientation="horizontal")
# cbar.set_label("density")
fig_vortector = plt.figure(dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [148]:
from synthvortex import VortexGenerator
scaling = 2
# vgen = VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling)
# vgen = VortexGenerator(3, periodic_y=True, Nr=121*scaling,Nphi=251*scaling, seed=123009799731638641813521844717530781350, noise=0)
vgen = VortexGenerator(filename="synthvort_examples/detection_overlapping_vortices.pickle")
vd = Vortector(vgen.Rs, vgen.Phis, vgen.Area, vgen.vortensity, vgen.density, verbose=True)
# n = 0
# vd.fit(n)
# for k in range(2):
#     vd.fit(n, region="combined") 
def detect_and_fit(vd):
    vd.detect(include_mask=True, autofit=False, keep_internals=False)
#     for k in range(len(vd.vortices)):
#         vd.fit(k)
#         vd.fit(k, region="combined")
%time detect_and_fit(vd)
n = None
show_fit_overview_2D(vd, n=n, bnd_pnts=False, fig=fig_vortector)
print("Number of vortices:", len(vd.vortices))
vgen.seed

Number of found contours: 93
Number of closed contours: 84
Area of contours: [14128.0, 14128.0, 12371.5, 12371.5, 11133.0, 11133.0, 10073.5, 10073.5, 9225.0, 9225.0, 8383.5, 8383.5, 4634.0, 4634.0, 4288.0, 4288.0, 3981.0, 3981.0, 3665.0, 3665.0, 3383.0, 3383.0, 3100.5, 3100.5, 3037.0, 3037.0, 2843.5, 2843.5, 2773.0, 2773.0, 2578.5, 2578.5, 2542.5, 2542.5, 2332.5, 2332.5, 2317.5, 2317.5, 2123.0, 2123.0, 2051.5, 2051.5, 1939.5, 1939.5, 1774.5, 1774.5, 1739.5, 1739.5, 1563.5, 1563.5, 1473.5, 1473.5, 1369.5, 1369.5, 1177.5, 1177.5, 1177.0, 1177.0, 977.5, 977.5, 901.0, 901.0, 780.0, 780.0, 608.0, 608.0, 582.0, 582.0, 374.0, 374.0, 253.5, 253.5, 252.5, 229.5, 229.5, 209.5, 209.5, 147.5, 147.5, 127.5, 97.5, 67.0, 18.0, 16.0]
Removed 17 candidates by common boundary area. 18 remaining
Removed 1 candidates by due to vortensity drop < 0.1.
CPU times: user 194 ms, sys: 96 µs, total: 194 ms
Wall time: 193 ms
Number of vortices: 2


123009799731638641813521844717530781350

In [119]:
for vort in vd.vortices:
    print(vort["contour"]["stats"]["vortensity_min"])

-0.13240203318373686
-0.08159054445480816


In [10]:
# vgen.save("synthvort_examples/detection_overlapping_vortices.pickle", 
#           note="An example in which the vortector puts two vortices into one contour.")

In [114]:
fig_single_vortex = plt.figure(dpi=150)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
def vort_sigma_plot(fig,Rs,Phis,vortensity,density,mask):
    fig.clf()
    gs = gridspec.GridSpec(ncols=2, nrows=1, figure=fig_single_vortex)
    axes = [fig.add_subplot(gs[0, 0]),fig.add_subplot(gs[0, 1])]

    vort_masked = np.ma.masked_array(vortensity, mask=~mask)
    im = axes[0].pcolormesh(Rs, Phis, vort_masked, vmin=0, vmax=1, shading="auto")
    axes[0].contour(Rs, Phis, vort_masked, cmap="Greys", linewidths=1)
    axes[0].set_xlabel("r")
    axes[0].set_ylabel("phi")
    cbar = fig.colorbar(im, ax=axes[0], orientation="horizontal")
    cbar.set_label("vortensity")

    density_masked = np.ma.masked_array(density, mask=~mask)
    im = axes[1].pcolormesh(Rs, Phis, density_masked, shading="auto")
    axes[1].contour(Rs, Phis, density_masked, cmap="Greys", linewidths=1)
    axes[1].set_xlabel("r")
    cbar = fig.colorbar(im, ax=axes[1], orientation="horizontal")
    cbar.set_label("density")

vort_data = vd.extract_data(vd.vortices[0], region="combined")
vort_sigma_plot(fig_single_vortex, *vort_data)

In [132]:
vd.vortices[1]["contour"]

{'detection': {'contour_value': 0.6500000000000001,
  'opencv_contour_number': 33000,
  'uuid': '22d071ef-fd81-4266-b62a-86332523251b',
  'pixel_area': 27883.5,
  'pixel_arcLength': 679.0853482484818,
  'bounding_x_img': array([167, 304], dtype=int32),
  'bounding_y_img': array([ 871, 1133], dtype=int32),
  'pnt_xlow_img': array([167, 985], dtype=int32),
  'pnt_xhigh_img': array([304, 986], dtype=int32),
  'pnt_ylow_img': array([235, 871], dtype=int32),
  'pnt_yhigh_img': array([ 255, 1133], dtype=int32),
  'dx_img': 137,
  'dy_img': 262,
  'ellipse': {'center_img': (1003.8944702148438, 237.67210388183594),
   'axesLengths_img': (136.90890502929688, 261.1641845703125),
   'angle_img': 93.65411376953125},
  'ellipse_area_delta_relative': 0.056115457565883015,
  'ellipse_area_delta': 1564.695361038299,
  'parameters': {'Nx': 363,
   'Ny': 753,
   'max_ellipse_aspect_ratio': inf,
   'max_ellipse_deviation': 0.15,
   'levels': [-1.0,
    -0.95,
    -0.9,
    -0.85,
    -0.8,
    -0.75,
   